In [1]:
import (
	"encoding/json"
    . "github.com/FactomProject/ptnet-eventstore/contract"
	"github.com/FactomProject/ptnet-eventstore/finite"
	. "github.com/FactomProject/ptnet-eventstore/identity"
	. "github.com/FactomProject/ptnet-eventstore/ptnet"
	"github.com/FactomProject/ptnet-eventstore/x"
)

In [2]:
// the state machine definition composes the rules of the contract
// machine must be designed to always end in a halting state
// this protocol requires the machine to be halted before tokens are released
//
// NOTE:
// this type of state machine classified as a https://en.wikipedia.org/wiki/Vector_addition_system
// and can also be represented graphically as a https://en.wikipedia.org/wiki/Petri_net
m := Machine{
    // define the inital state - a nice convention follow
    // is to set each variable to the max possible value
    // that can be achieved during valid contract execution
    // in this contract no value ever exceeds 1
	//                   00 01 02 10 11 12 20 21 22 O  X $O $X $DEP
	Initial: StateVector{1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1},

	Transitions: map[string]Transition{
        // each transition defines its effect 
        // on every offset 'variable' declared in the state vector
        //
        // NOTE: this is mathematical vector addition at work
        // each vector is 'executed' by adding it to the preceding state
		//                00 01 02 10 11 12 20 21 22  O  X $O $X $DEP
        // by convention a contract must have a BEGIN/"EXEC" transaction
        // this action is executed when the contract is first offered
		BEGIN: Transition{0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, -1, -1, -1},
        // moves for player X
		"X00": Transition{-1, 0, 0, 0, 0, 0, 0, 0, 0, 1, -1, 0, 0, 0},
		"X01": Transition{0, -1, 0, 0, 0, 0, 0, 0, 0, 1, -1, 0, 0, 0},
		"X02": Transition{0, 0, -1, 0, 0, 0, 0, 0, 0, 1, -1, 0, 0, 0},
		"X10": Transition{0, 0, 0, -1, 0, 0, 0, 0, 0, 1, -1, 0, 0, 0},
		"X11": Transition{0, 0, 0, 0, -1, 0, 0, 0, 0, 1, -1, 0, 0, 0},
		"X12": Transition{0, 0, 0, 0, 0, -1, 0, 0, 0, 1, -1, 0, 0, 0},
		"X20": Transition{0, 0, 0, 0, 0, 0, -1, 0, 0, 1, -1, 0, 0, 0},
		"X21": Transition{0, 0, 0, 0, 0, 0, 0, -1, 0, 1, -1, 0, 0, 0},
		"X22": Transition{0, 0, 0, 0, 0, 0, 0, 0, -1, 1, -1, 0, 0, 0},
        // moves for player O
		"O00": Transition{-1, 0, 0, 0, 0, 0, 0, 0, 0, -1, 1, 0, 0, 0},
		"O01": Transition{0, -1, 0, 0, 0, 0, 0, 0, 0, -1, 1, 0, 0, 0},
		"O02": Transition{0, 0, -1, 0, 0, 0, 0, 0, 0, -1, 1, 0, 0, 0},
		"O10": Transition{0, 0, 0, -1, 0, 0, 0, 0, 0, -1, 1, 0, 0, 0},
		"O11": Transition{0, 0, 0, 0, -1, 0, 0, 0, 0, -1, 1, 0, 0, 0},
		"O12": Transition{0, 0, 0, 0, 0, -1, 0, 0, 0, -1, 1, 0, 0, 0},
		"O20": Transition{0, 0, 0, 0, 0, 0, -1, 0, 0, -1, 1, 0, 0, 0},
		"O21": Transition{0, 0, 0, 0, 0, 0, 0, -1, 0, -1, 1, 0, 0, 0},
		"O22": Transition{0, 0, 0, 0, 0, 0, 0, 0, -1, -1, 1, 0, 0, 0},

        // moves that halt game and unlock tokens for winner
        // or refund to the depositor
		"WINX": Transition{0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 1, 0},
		"WINO": Transition{0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 1, 0, 0},
		"ENDX": Transition{0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 1},
		"ENDO": Transition{0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 1},
	},
}

In [3]:
// contract declaration adds more rules in addition to the state machine
gameContract := Declaration{
    // inputs are similar to most cryptocurrency rules
    // and represent a type of 'pay-to-script' functionality
    // tokens are lock away in the contract until the blockheight is exceeded (invalid/fault)
    // or the contract halts (valid completion)
	Inputs: []AddressAmountMap{
		AddressAmountMap{Address[DEPOSITOR], 1}, // deposit 1 token
	},
    // outputs are referenced by Guards and Conditions
    // and are used as an Access Control List (ACL)
    // to define roles of users that are allowed to interact
    // with the contract in addition to defining which users
    // can withdraw tokens from a completed/halted contract
	Outputs: []AddressAmountMap{ // array of possible redeemers
		AddressAmountMap{Address[DEPOSITOR], 1},
		AddressAmountMap{Address[PLAYERX], 1},
		AddressAmountMap{Address[PLAYERO], 1},
	},
	BlockHeight: 60221409,      // deadline to achieve halting state
	Salt:        "|RANDOM|",    // random salt
	ContractID:  "|OctoeContractID|",// unique ID for this contract instance
	Schema:      "octoe-v1",    // versioned contract schema
	State:       m.Initial,     // state machine initial state
	Actions:     m.Transitions, // state machine defined transitions
	Guards: []Condition{ // guards restrict state machine actions
		//       00 01 02 10 11 12 20 21 22  O  X $O $X $DEP
		Condition{0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0}, // Admin/Depositor
		Condition{0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0},// PlayerX turn
		Condition{0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0},// PlayerO turn
	},
	Conditions: []Condition{ // redeem conditions to unlock input tokens
		//       00 01 02 10 11 12 20 21 22  O  X $O $X $DEP
		Condition{0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1},// no win/refund deposit
		Condition{0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0},// X win
		Condition{0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0},// O win 
	},
}

In [4]:
// use declaration to construct an offer
// this is a data wrapper that is used to isolate transaction ops from declaration ops
// NOTE: mostly this is just an artifact of this implementation not especially meaningful
offer := finite.Offer{
	Declaration: gameContract,
}

In [5]:
[]interface{}{gameContract.ContractID, gameContract.Schema, gameContract.BlockHeight}

[|OctoeContractID| octoe-v1 60221409]

In [6]:
// check to see if contract has been recorded yet (it hasn't)
Exists(gameContract.Schema, gameContract.ContractID)

false

In [7]:
// to begin contract execution an - offer transaction is made
// by convention the initial state should be set to the upper bound
// for all the variables used by the contract
// in this contract no variable is ever > 1
//
// NOTE by design - the state machine reports any output values < 0 as invalid
// see: https://en.wikipedia.org/wiki/Vector_addition_system for more details
txn := finite.OfferTransaction(offer, Private[DEPOSITOR])
[]interface{}{txn.Action, txn.InputState, "=>", txn.OutputState}

[EXEC [1 1 1 1 1 1 1 1 1 1 1 1 1 1] => [1 1 1 1 1 1 1 1 1 0 1 0 0 0]]

In [8]:
// check to see if our contract has been recorded
Exists(gameContract.Schema, gameContract.ContractID)

true

In [9]:
// show that contract is still open
IsHalted(gameContract)

false

In [10]:
// show that tokens are locked
// even original depositor cannot withdraw tokens
CanRedeem(gameContract, Public[DEPOSITOR])

false

In [11]:
// Helper function to make it easier to compose transactions in this example
func commit(action string, key PrivateKey, payload []byte) (finite.Transaction, error) {
	pub := PublicKey{}
	copy(pub[:], x.PrivateKeyToPub(key[:])) // get public key
	return finite.ExecuteTransaction(finite.Execution{
		Command: Command{
			ChainID:    CHAIN_ID,// which chain to add entry
			ContractID: "|OctoeContractID|",
			Schema:     OctoeV1, // state machine version
			Action:     action,  // state machine action
			Amount:     1,       // triggers input action 'n' times amount
			Payload:    payload, // arbitrary data optionally included
			Pubkey:     pub,     // must match key in contract declaration
		},
	}, key)
}

In [12]:
// an event with an arbitrary payload attached
// on-chain it is expected that this payload is used to attach
// additional supporting data or evidence that 
// an action is being executed according to the intent of the contract
payload, _ := json.Marshal([]string{"hello", "world"})
event1, _ := commit("X11", Private[PLAYERX], payload)

[]interface{}{event1.Action, event1.InputState, "=>", event1.OutputState}

[X11 [1 1 1 1 1 1 1 1 1 0 1 0 0 0] => [1 1 1 1 0 1 1 1 1 1 0 0 0 0]]

In [13]:
// show an invalid move
// according to this state machine players must take turns
event2, err := commit("X00", Private[PLAYERX], nil)
[]interface{}{event2.Action, event2.InputState, "=>", err}

[X00 [1 1 1 1 0 1 1 1 1 1 0 0 0 0] => invalid output: -1 offset: 10]

In [14]:
// another invalid move - here we try to sign with an incorrect key
// contract makes players sign each event
event3, err := commit("O01", Private[PLAYERX], nil)

[]interface{}{event3.Action, event3.InputState, "=>", err}

[O01 [1 1 1 1 0 1 1 1 1 1 0 0 0 0] => failed guard condition]

In [15]:
// valid
event4, err := commit("O01", Private[PLAYERO], nil)

[]interface{}{event4.Action, event4.InputState, "=>", event4.OutputState}

[O01 [1 1 1 1 0 1 1 1 1 1 0 0 0 0] => [1 0 1 1 0 1 1 1 1 0 1 0 0 0]]

In [16]:
// invalid - state machine ensures move is used only once
// move X11 is already taken as seen above
event5, err := commit("X11", Private[PLAYERX], nil)

[]interface{}{event5.Action, event5.InputState, "=>", err}

[X11 [1 0 1 1 0 1 1 1 1 0 1 0 0 0] => invalid output: -1 offset: 4]

In [17]:
// valid
event6, err := commit("X00", Private[PLAYERX], nil)

[]interface{}{event6.Action, event6.InputState, "=>", event6.OutputState}

[X00 [1 0 1 1 0 1 1 1 1 0 1 0 0 0] => [0 0 1 1 0 1 1 1 1 1 0 0 0 0]]

In [18]:
// valid
event7, err := commit("O02", Private[PLAYERO], nil)

[]interface{}{event7.Action, event7.InputState, "=>", event7.OutputState}

[O02 [0 0 1 1 0 1 1 1 1 1 0 0 0 0] => [0 0 0 1 0 1 1 1 1 0 1 0 0 0]]

In [19]:
// valid
event8, err := commit("X22", Private[PLAYERX], nil)

[]interface{}{event8.Action, event8.InputState, "=>", event8.OutputState}

[X22 [0 0 0 1 0 1 1 1 1 0 1 0 0 0] => [0 0 0 1 0 1 1 1 0 1 0 0 0 0]]

In [20]:
// game is over
// contract depositor confirms the winner
// this halts the state machine thereby unlocking the deposited tokens
event9, err := commit("WINX", Private[DEPOSITOR], nil)

[]interface{}{event9.Action, event9.InputState, "=>", event9.OutputState}

[WINX [0 0 0 1 0 1 1 1 0 1 0 0 0 0] => [0 0 0 1 0 1 1 1 0 0 0 0 1 0]]

In [21]:
// contract is only redeemable after halting
IsHalted(gameContract)

true

In [22]:
// only the winner can redeem the deposit
CanRedeem(gameContract, Public[PLAYERX])

true

In [23]:
// loser has no claim
CanRedeem(gameContract, Public[PLAYERO])

false